# Import the pymongo Library

The first step in using `pymongo` is to import it into our local context.

This exposes all methods exported from the `pymongo` library and makes them available in our local context.

For additional information on all methods available in `pymongo`, visit the [documentation](https://api.mongodb.com/python/current/)

In [1]:
%pwd # should be mflix-python
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymongo

# Establish a connection

When working with `pymongo` is to create a client that exposes the connection to your MongoDB instance. Since we're working with MongoDB Atlas, you'll need the connection URI - you can obtain this by clicking the _CONNECT_ button from within the Atlas Management Console. 

![Connect](atlas_connect.gif)


###  Create a variable to hold the URI String which points to our MongoDB Atlas Cluster. 

Note the use of the SRV Record. This simplifies reference to a cluster of nodes.

In [12]:
uri = "mongodb+srv://<Your User>:<Password>@mflix-zbcul.mongodb.net/?retryWrites=true&w=majority"

### Now, create a client object that will expose methods from our pymongo MongoClient connection.

In [13]:
client = pymongo.MongoClient(uri)

### Create a variable that points to the database from our client connection.

In this case, we\'re accessing our `sample_mflix` collection in our cluster. If you have not loaded the sample databases, review this document.


In [14]:
mflix = client.sample_mflix_test

### Create a variable pointing to the collection within our mflix database

In [15]:
movies = mflix.movies

# CREATE: Creating, or Inserting Data into MongoDB

Now that we have a connection, let's Create a new document in the movies connection

[Documentation for Creating](https://docs.mongodb.com/manual/crud/#create-operations)

## Inserting a Single Document

In [16]:
movies.insert_one({
    'title': 'a new movie',
    'fullplot': 'a new movie about a panda that eats shoots and leaves',
    'countries': ['USA'],
     'year': 1893,
     'type': 'movie',
     'genres': ['Short'],
     'cast': ['Panda', 'Michael Scott']
})

## Inserting Many Documents
Note that you can simply create an array of JSON documents and pass that to the insert_many method.

In [17]:
moremovies = [
    {
        "title": "a title",
        'fullplot': 'a movie',
        'countries': ['USA'],
        'year': 1891,
         'type': 'movie',
         'genres': ['Short'],
         'cast': ['Chad Smith', 'Anthony Keidis']
    },{
        "title": "b title",
        'fullplot': 'b movie',
        'countries': ['USA'],
        'year': 1894,
         'type': 'movie',
         'genres': ['Short'],
         'cast': ['Getty Lee', 'Neil Pert']
    },{
        "title": "c title",
        'fullplot': 'c movie',
        'countries': ['USA'],
        'year': 1892,
         'type': 'movie',
         'genres': ['Short'],
         'cast': ['Sasha Baron Cohen']
    }
]
movies.insert_many(moremovies)


# READ: Reading Documents with Pymongo

[Documentation for Reads](https://docs.mongodb.com/manual/crud/#read-operations)

With relational, or _tabular_ databases, to "select" all "rows" in a "table", we would:

```SELECT * FROM movies```

The terminology differs slightly - but the elements in data persistence are quite similar.

## Database Elements

|Tabular|Document|
|-------|-------|
|table|collection|
|row|document|
|column|key|
|index|index|

## Data Query Commands
|SQL|MQL|
|---|---|
|select|find|

With that in mind, To "find" all documents in the collection, pass an empty document as the query filter parameter to the find method. The query filter parameter determines the select criteria:


In [18]:
import pprint
cursor = movies.find({}).limit(10)
for document in cursor:
    pprint.pprint(document) # iterate the cursor

{'_id': ObjectId('5d235693b0b6f9a9deb0f046'),
 'cast': ['Panda', 'Michael Scott'],
 'countries': ['USA'],
 'fullplot': 'a new movie about a panda that eats shoots and leaves',
 'genres': ['Short'],
 'title': 'a new movie',
 'type': 'movie',
 'year': 1893}
{'_id': ObjectId('5d2356d4b0b6f9a9deb0f047'),
 'cast': ['Chad Smith', 'Anthony Keidis'],
 'countries': ['USA'],
 'fullplot': 'a movie',
 'genres': ['Short'],
 'title': 'a title',
 'type': 'movie',
 'year': 1891}
{'_id': ObjectId('5d2356d4b0b6f9a9deb0f048'),
 'cast': ['Getty Lee', 'Neil Pert'],
 'countries': ['USA'],
 'fullplot': 'b movie',
 'genres': ['Short'],
 'title': 'b title',
 'type': 'movie',
 'year': 1894}
{'_id': ObjectId('5d2356d4b0b6f9a9deb0f049'),
 'cast': ['Sasha Baron Cohen'],
 'countries': ['USA'],
 'fullplot': 'c movie',
 'genres': ['Short'],
 'title': 'c title',
 'type': 'movie',
 'year': 1892}


## find_one returns a Python dictionary from the database

In [19]:
movies.find_one({'title': 'a title'})

{'_id': ObjectId('5d2356d4b0b6f9a9deb0f047'),
 'title': 'a title',
 'fullplot': 'a movie',
 'countries': ['USA'],
 'year': 1891,
 'type': 'movie',
 'genres': ['Short'],
 'cast': ['Chad Smith', 'Anthony Keidis']}

In [20]:
import pprint
pprint.pprint(movies.count_documents({"title": "a title"}))

1


## Querying by Document Attributes

Let's say our document model contains an array and we want to find all documents where an element in that array matches a specific value.  

Our movie document model contains several array fields - Cast, for example is an array of all members of the cast for a given movie.  How might we find all movies where a specific actor was in the cast?
```
{
   title: "a movie title",
   cast: ["Sasha Baron Cohen"]
}

```

How might we find all documents where `John Smith` is in the cast?

To find elements that exist in a document array - use the `$in` operator as part of the `find` command.  Review the [documentation](https://docs.mongodb.com/manual/reference/operator/query/in/) for more information.



## find returns a cursor - you can iterate over this like any other iterable

In [27]:
cursor = movies.find({ "cast" : { "$in" : ["Sasha Baron Cohen" ] }})
list(cursor)

[{'_id': ObjectId('5d2356d4b0b6f9a9deb0f049'),
  'title': 'c title',
  'fullplot': 'c movie',
  'countries': ['USA'],
  'year': 1892,
  'type': 'movie',
  'genres': ['Short'],
  'cast': ['Sasha Baron Cohen']}]

## Projections: Controlling what fields are displayed.

By default, queries in MongoDB return all fields in matching documents. To limit the amount of data that MongoDB sends to applications, you can include a [projection document](https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/) to specify or restrict fields to return.

To list out the results of our find with a projection... we can use list()

In [25]:
cursor =  movies.find(
        { "cast" : { "$in" : ["Sasha Baron Cohen"] } }, 
        projection = { "title" : 1 }
    )

list( cursor )

[{'_id': ObjectId('5d2356d4b0b6f9a9deb0f049'), 'title': 'c title'}]

# UPDATE: Update our movie document
[Documentation for Updates](https://docs.mongodb.com/manual/tutorial/update-documents/)

In [21]:
find_query = { "title": "a title" }

new_writers = { "writers": [ "michael lynn", "matthew javaly", "maxime beugnet" ] } 

movies.update_one( find_query, { "$set": new_writers })

movies.find_one({ "title": "a new movie" })


{'_id': ObjectId('5d1f2d77bb54326175210e76'),
 'title': 'a new movie',
 'fullplot': 'a new movie about a panda that eats shoots and leaves',
 'countries': ['USA'],
 'year': 1893,
 'type': 'movie',
 'genres': ['Short'],
 'cast': ['Panda', 'Michael Scott'],
 'writers': ['michael lynn', 'matthew javaly', 'maxime beugnet']}

# DELETE: Deleting Documents
[Documentation for Deletes](https://docs.mongodb.com/manual/tutorial/remove-documents/)

## Deleting one at a time

In [30]:
movies.delete_one({"title": "a title"})

In [29]:
# find it?
movies.find_one({"title": "a title"})

## Deleting many

In [31]:
myquery = { "title": {"$regex": "^[abc] title"} }
x = movies.delete_many(myquery)
print(x.deleted_count, " movies deleted.")


12  movies deleted.
